In [3]:
import json
import os
import pickle
import numpy as np
from collections import Counter

from multiprocessing import Pool

import tqdm

In [3]:
os.chdir('/data')

In [4]:
# 구글드라이브 DATA에 데이터있음
with open('no_pos_review.json') as fp:
    review = json.load(fp)
with open('voca_freq_dic_new.json') as fp:
    voca_freq_dic = json.load(fp)

In [26]:
# key_lst = list(review.keys())
# sampled_key_lst = key_lst[:3000]
# review2 = {}
# for k in sampled_key_lst:
#     review2[k] = review[k]
# len(review2)

In [40]:
voca_freq_dic2 = {} # freq로 걸러서 일부 단어만 사용
for k,v in voca_freq_dic.items():
    if v>99:
        voca_freq_dic2[k] = v

In [41]:
len(voca_freq_dic2)

4254

In [122]:
# sorted(voca_freq_dic2.items(),key=lambda kv:kv[1],reverse=False)

In [43]:
# LDA 결과(토픽별 키워드 리스트)를 aspect key words로 갖다써보자
# 현재 불용어처리 등 덜된상태
tmp = {0: [('station_N', 0.075440474),
  ('convenient_N', 0.035710342),
  ('location_N', 0.033328794),
  ('walk_V', 0.03065599),
  ('subway_N', 0.026240626),
  ('place_N', 0.023897689),
  ('minute_N', 0.023682337),
  ('easy_J', 0.022524938),
  ('locate_V', 0.019995727),
  ('store_N', 0.019815892),
  ('close_J', 0.019692997),
  ('close_R', 0.019388903),
  ('away_R', 0.01791159),
  ('bus_N', 0.017175121),
  ('airport_N', 0.0151694175),
  ('convenience_N', 0.014715803),
  ('also_R', 0.014201972),
  ('walk_N', 0.013559201),
  ('subway_J', 0.01225483),
  ('apartment_N', 0.01212426),
  ('convenient_J', 0.01193456),
  ('go_V', 0.010800422),
  ('distance_N', 0.010039834),
  ('find_V', 0.009765462),
  ('subway_R', 0.009758889),
  ('take_V', 0.009583443),
  ('get_V', 0.009291667),
  ('line_N', 0.009158479),
  ('restaurant_N', 0.0081279725),
  ('house_N', 0.007849644)],
 1: [('good_J', 0.09984607),
  ('nice_J', 0.0702112),
  ('location_N', 0.065345466),
  ('really_R', 0.06501797),
  ('place_N', 0.053646892),
  ('comfortable_J', 0.033898752),
  ('clean_J', 0.03378657),
  ('stay_N', 0.025737181),
  ('house_N', 0.025186999),
  ('accommodation_N', 0.016483787),
  ('apartment_N', 0.016000627),
  ('perfect_J', 0.015435147),
  ('space_N', 0.01426183),
  ('bed_N', 0.0142121855),
  ('wifi_N', 0.013773334),
  ('landlord_N', 0.010759912),
  ('perfect_N', 0.010674935),
  ('amenity_N', 0.009882116),
  ('new_J', 0.009253663),
  ('tidy_N', 0.008537611),
  ('quiet_J', 0.008063444),
  ('lovely_R', 0.007641677),
  ('central_J', 0.0074305693),
  ('pretty_R', 0.007232856),
  ('neat_N', 0.007133313),
  ('egg_N', 0.007132264),
  ('expect_V', 0.0070267115),
  ('photo_N', 0.0068760742),
  ('facility_N', 0.006667349),
  ('felt_V', 0.006483669)],
 2: [('area_N', 0.06683145),
  ('night_N', 0.035611134),
  ('quiet_J', 0.030158201),
  ('quite_R', 0.023574363),
  ('view_N', 0.023483817),
  ('price_N', 0.022715302),
  ('bit_N', 0.021813327),
  ('see_V', 0.02036446),
  ('neighborhood_N', 0.019612567),
  ('right_J', 0.017915798),
  ('little_J', 0.01292769),
  ('city_N', 0.012332531),
  ('part_N', 0.011145581),
  ('apartment_N', 0.010512025),
  ('beautiful_J', 0.010445903),
  ('locate_V', 0.010315028),
  ('park_N', 0.009846657),
  ('building_N', 0.009445184),
  ('issue_N', 0.008640434),
  ('reply_V', 0.008343354),
  ('soon_R', 0.008054228),
  ('hard_J', 0.008007643),
  ('window_N', 0.007892024),
  ('also_R', 0.0077502076),
  ('open_J', 0.007630731),
  ('door_N', 0.007591834),
  ('especially_R', 0.0068424055),
  ('get_V', 0.006388696),
  ('house_N', 0.006340781),
  ('perfect_V', 0.0062577077)],
 3: [('great_J', 0.15104426),
  ('home_N', 0.03838182),
  ('experience_N', 0.034535814),
  ('thank_N', 0.03342067),
  ('stay_N', 0.022656428),
  ('make_V', 0.01760778),
  ('thing_N', 0.01690892),
  ('airbnb_N', 0.015427705),
  ('place_V', 0.014597721),
  ('feel_N', 0.014560298),
  ('overall_J', 0.013511427),
  ('super_J', 0.012852805),
  ('apartment_N', 0.0127586955),
  ('feel_V', 0.012492993),
  ('instruction_N', 0.011910272),
  ('know_V', 0.009106032),
  ('flat_J', 0.009034321),
  ('house_N', 0.008116479),
  ('ive_J', 0.008065549),
  ('anything_N', 0.0076171067),
  ('meet_N', 0.007522718),
  ('value_N', 0.0071481066),
  ('book_V', 0.00696656),
  ('choice_N', 0.0067781466),
  ('different_J', 0.0067624864),
  ('reply_N', 0.0065802587),
  ('never_R', 0.0061996435),
  ('hospitality_N', 0.006097631),
  ('jay_N', 0.0060664965),
  ('bad_J', 0.0058132866)],
 4: [('room_N', 0.112217285),
  ('well_R', 0.051788304),
  ('everything_N', 0.051343787),
  ('clean_J', 0.050989766),
  ('house_N', 0.025484154),
  ('apartment_N', 0.02400923),
  ('need_V', 0.022365913),
  ('clean_N', 0.021655692),
  ('small_J', 0.02147159),
  ('comfortable_J', 0.01924563),
  ('need_N', 0.017338376),
  ('cozy_N', 0.016669074),
  ('big_J', 0.014753495),
  ('spacious_J', 0.013613145),
  ('bathroom_N', 0.012964344),
  ('nice_N', 0.01125928),
  ('people_N', 0.01100092),
  ('picture_N', 0.010688399),
  ('provide_V', 0.010524188),
  ('warm_J', 0.00983317),
  ('kitchen_N', 0.009782993),
  ('clean_V', 0.009278437),
  ('also_R', 0.008697552),
  ('responsive_J', 0.008031375),
  ('towel_N', 0.007448198),
  ('large_J', 0.0072318553),
  ('bed_V', 0.0071571474),
  ('equip_V', 0.0070687374),
  ('enough_R', 0.0065676877),
  ('live_V', 0.006433929)],
 5: [('even_R', 0.023927504),
  ('floor_N', 0.020947738),
  ('get_V', 0.016879946),
  ('little_J', 0.01670006),
  ('problem_N', 0.016578786),
  ('water_N', 0.016478194),
  ('exactly_R', 0.013305269),
  ('extremely_R', 0.0122868465),
  ('didnt_N', 0.011868971),
  ('use_N', 0.01179084),
  ('machine_N', 0.011563873),
  ('show_V', 0.011448914),
  ('night_N', 0.011121356),
  ('wash_V', 0.010934781),
  ('late_J', 0.010417275),
  ('check_V', 0.010238513),
  ('hot_J', 0.010211995),
  ('use_V', 0.010035234),
  ('however_R', 0.009700446),
  ('arrive_V', 0.009524829),
  ('accommodate_V', 0.009023434),
  ('early_J', 0.0089837825),
  ('also_R', 0.008448831),
  ('air_N', 0.008398446),
  ('check_N', 0.008317534),
  ('still_R', 0.00801234),
  ('leave_V', 0.007958689),
  ('toilet_N', 0.0075545954),
  ('bed_N', 0.0074843178),
  ('find_V', 0.0071834484)],
 6: [('day_N', 0.044497136),
  ('make_V', 0.035028186),
  ('time_N', 0.02947009),
  ('friend_N', 0.023036933),
  ('much_J', 0.022923442),
  ('stay_V', 0.020721437),
  ('help_N', 0.020701654),
  ('love_V', 0.020222116),
  ('trip_N', 0.018225666),
  ('go_V', 0.018032515),
  ('first_J', 0.017838366),
  ('family_N', 0.016667131),
  ('wonderful_J', 0.01531437),
  ('sure_J', 0.012547571),
  ('quickly_R', 0.012029172),
  ('first_R', 0.010693285),
  ('im_N', 0.010558216),
  ('welcome_V', 0.010431975),
  ('house_N', 0.009905076),
  ('night_N', 0.009419678),
  ('get_V', 0.009239833),
  ('way_N', 0.009100049),
  ('happy_J', 0.008998546),
  ('even_R', 0.00883439),
  ('luggage_N', 0.008800614),
  ('recommended_J', 0.008401327),
  ('respond_V', 0.008349242),
  ('second_J', 0.008234377),
  ('able_J', 0.008000888),
  ('chance_N', 0.007491548)]
       ,
 7: [('lot_N', 0.043792848),
  ('restaurant_N', 0.040813655),
  ('many_J', 0.03109465),
  ('food_N', 0.029509438),
  ('good_J', 0.025894398),
  ('take_V', 0.024536517),
  ('min_N', 0.024025736),
  ('street_N', 0.021218892),
  ('shop_N', 0.020949543),
  ('there_N', 0.015530614),
  ('korean_J', 0.015057178),
  ('person_N', 0.014282671),
  ('shop_V', 0.0127358185),
  ('studio_N', 0.012632791),
  ('also_R', 0.012036752),
  ('excellent_J', 0.011713238),
  ('transportation_N', 0.011573859),
  ('coffee_N', 0.011001995),
  ('fantastic_J', 0.01008252),
  ('nearby_R', 0.010082044),
  ('local_J', 0.009666101),
  ('warm_N', 0.009533902),
  ('rest_N', 0.009117627),
  ('care_N', 0.009051847),
  ('bar_N', 0.008858041),
  ('university_N', 0.0079645645),
  ('traditional_J', 0.0073084603),
  ('life_N', 0.007216505),
  ('house_N', 0.0067656073),
  ('thing_N', 0.0066509987)],
 8: [('place_N', 0.09395834),
  ('stay_V', 0.06079642),
  ('stay_N', 0.05158205),
  ('time_N', 0.04721144),
  ('recommend_V', 0.041916873),
  ('come_V', 0.03651728),
  ('definitely_R', 0.036476232),
  ('next_J', 0.03116234),
  ('back_R', 0.029059581),
  ('go_V', 0.023483075),
  ('thanks_N', 0.019712204),
  ('want_V', 0.019274745),
  ('visit_N', 0.01906872),
  ('highly_R', 0.017565712),
  ('enjoy_V', 0.01718251),
  ('best_J', 0.010691537),
  ('amaze_V', 0.010670577),
  ('stay_J', 0.010333447),
  ('house_N', 0.010283317),
  ('apartment_N', 0.009439452),
  ('stop_N', 0.009254608),
  ('look_V', 0.008955529),
  ('guesthouse_N', 0.0072605656),
  ('anyone_N', 0.007109243),
  ('youre_N', 0.0068305926),
  ('trip_N', 0.006618274),
  ('book_N', 0.006315217),
  ('visit_V', 0.0062876446),
  ('use_N', 0.0058828676),
  ('week_N', 0.0058368403)],
 9: [('host_N', 0.0983368),
  ('kind_N', 0.034277067),
  ('helpful_J', 0.025665222),
  ('help_V', 0.022550533),
  ('give_V', 0.020565689),
  ('also_R', 0.020454295),
  ('nice_J', 0.020182502),
  ('really_R', 0.01883619),
  ('provide_V', 0.018348224),
  ('question_N', 0.017101107),
  ('friendly_R', 0.01532856),
  ('always_R', 0.014121552),
  ('friendly_J', 0.013314891),
  ('helpful_N', 0.012839154),
  ('guest_N', 0.012607242),
  ('owner_N', 0.011304215),
  ('breakfast_N', 0.010966653),
  ('even_R', 0.009854197),
  ('recommend_J', 0.009247552),
  ('quick_J', 0.009128347),
  ('communication_N', 0.008465741),
  ('meet_V', 0.007926063),
  ('information_N', 0.007655206),
  ('get_V', 0.007644172),
  ('direction_N', 0.0075688995),
  ('easy_J', 0.0075597554),
  ('need_V', 0.006919648),
  ('shopping_N', 0.0067448863),
  ('free_J', 0.006327245),
  ('staff_N', 0.005882458)]
      }

In [44]:
seed_words = {}
for k in range(len(tmp)):
    seed_words[k] = []
    for idx,tpl in enumerate(tmp[k]):
        seed_words[k].append( (tpl[0][:-2],tpl[1]) )
    

In [47]:
# seed_words

In [46]:
aspect_keywords = []
for i,k_words in seed_words.items():
    keys,b = zip(*k_words)
    aspect_keywords.append(keys)


In [48]:
def get_aspect_terms(aspects, vocab_dict):
    aspect_terms = []
    w_notfound = []
    
    for aspect_kws in aspects:
        aspect = []
        for w in aspect_kws:
            if w in vocab_dict:
                aspect.append(w)
            else:
                w_notfound.append(w)
        aspect_terms.append(aspect)
    #We are only using one hotel review file, as we keep inceasing the number of files words not found will decrease.
    # print "Words not found in vocab:", ' '.join(w_notfound)
    return aspect_terms

In [49]:
def chi_sq(a,b,c,d):
    c1 = a
    c2 = b - a
    c3 = c - a
    c4 = d - b - c + a
    nc =  d
    return nc * (c1*c4 - c2*c3) * (c1*c4 - c2*c3)/((c1+c3) * (c2+c4) * (c1+c2) * (c3+c4))

In [50]:
def chi_sq_mat():
    global aspect_words, aspect_sent, num_words
    asp_rank = np.zeros(aspect_words.shape)
    for i in range(len(aspect_terms)):
        for j in range(len(vocab)):
            asp_rank[i][j] = chi_sq(aspect_words[i][j], num_words[j], aspect_sent[i], len(sent))
    return asp_rank

In [51]:
aspect_terms = get_aspect_terms(aspect_keywords,voca_freq_dic2)

In [52]:
def aspect_segmentaion(review_dic,voca_freq_dic):
    #INPUT
    review_sent = [] # 문장, 단어단위로 토크나이징된 리뷰 담은 리스트
    for rev_lst in review_dic.values():
        for rev in rev_lst:
            review_sent.append(rev[1])
    
    all_ratings = []
    for rev_lst in review_dic.values():
        for rev in rev_lst:
            all_ratings.append(rev[0])

    #selection threshold
    p = 5
    
    #Iterations 
    # I = 10
    I = 1

#     #Create Vocabulary
#     #review_sent, review_actual, only_sent = parse_to_sentence(reviews)
#     #vocab, #vocab_dict = create_vocab(only_sent)

    vocab = list(voca_freq_dic.keys())
    #Assign a number corresponding to each word. Makes counting easier.
    vocab_dict = dict(zip(vocab, range(len(vocab)))) # word를 key 로 word의 인덱스를 value로  

    #Aspect Keywords
    aspect_terms = get_aspect_terms(aspect_keywords,voca_freq_dic)

#     label_text = 
    # print aspect_terms

    #ALGORITHM
    review_labels = []
    k = len(aspect_terms) # k: 토픽 개수 10
    v = len(vocab) # v: 단어 개수 18923
    
    aspect_words = np.zeros((k,v))
    aspect_sent = np.zeros(k)
    num_words = np.zeros(v)
    #-----------------------------------------------------
    for _ in range(I):
        # print(len(review_sent)) 리뷰개수 595387
        for r in review_sent:
            labels = []
            for s in r:
                count = np.zeros(len(aspect_terms)) # 한 문장에 대해 aspect keyword가 몇개씩있는지 카운트
                
                for i,a in enumerate(aspect_terms):
                    for w in s:
                        if w in vocab_dict:
                            num_words[vocab_dict[w]] += 1
                            if w in a:
                                count[i] += 1

                if max(count) > 0:
                    la = np.where(np.max(count) == count)[0].tolist() # count 중 max인 aspect들의  인덱스 뽑아 리스트로
                    
                    labels.append(la)
                    for idx in la:
                        aspect_sent[idx] += 1
                        for w in s:
                            if w in vocab_dict:
                                aspect_words[idx][vocab_dict[w]] += 1
                else:
                    labels.append([])
                    
            review_labels.append(labels)
        
#             aspect_w_rank = chi_sq_mat()
#             new_labels = []
#             for na in aspect_w_rank:
#                 x = np.argsort(na)[::-1][:p]
#                 new_labels.append(x)
#                 for k,v in vocab_dict.items():
#                     if vocab_dict[k] in x:
#                         print(k)
#             sys.exit()
            
    return review_labels,review_sent

#     ratings_sentiment = []
#     for r in review_actual:
#         sentiment = []
#         #aspect ratings based on sentiment
#         for s in r:
#             ss = sid.polarity_scores(s)
#             sentiment.append(ss['compound'])
#         ratings_sentiment.append(sentiment)

#     #Aspect Ratings Per Review
#     aspect_ratings = []
#     for i,r in enumerate(review_labels):
#         rating = np.zeros(7)
#         count = np.zeros(7)
#         rs = ratings_sentiment[i] 
#         for j,l in enumerate(r):
#             for k in range(7):
#                 if k in l:
#                     rating[k] += rs[j]
#             for k in range(7):
#                 if count[k] != 0:
#                     rating[k] /= count[k]
#         #Map from -[-1,1] to [1,5]
#         for k in range(7):
#             if rating[k] != 0:
#                 rating[k] = int(round((rating[k]+1)*5/2))
#         aspect_ratings.append(rating)
#     return aspect_ratings, all_ratings

    # n = 0
    # print review_actual[n], '\n', review_labels[n]
    # print ratings_sentiment[n], '\n', aspect_ratings[n]
    # print len(all_ratings), len(reviews), all_ratings[0]
    # sys.exit()
    # return aspect_ratings

    # print sent[5:9], labels[5:9]
    # print zip(actual_sent, labels)[:10]
    # print zip(actual_sent, sentiment)[:10]

In [53]:
# omit_word = []
# for k,rev_lst in review.items():
#     for rev in rev_lst:
#         for i,sent in enumerate(rev[1]):
#             sent_tmp =[]
#             for word in sent:
#                 if word in voca_freq_dic2:
#                     sent_tmp.append(word)
#                 else:
#                     omit_word.append(word)
#             rev[1][i] = sent_tmp
            

In [54]:
%%time
review_labels,review_sents = aspect_segmentaion(review,voca_freq_dic2)

CPU times: user 2min 32s, sys: 1.98 s, total: 2min 34s
Wall time: 2min 31s


In [64]:
# review_labels_np = []
# for rev in review_labels:
#     rev_np = []
#     for sent in rev:
#         rev_np.append(np.array(sent))
#     review_labels_np.append(np.array(rev_np))
# review_labels_np = np.array(review_labels_np)

In [68]:
# review_labels_np[0]

array([array([6]), array([4, 5, 6, 8]), array([0, 3, 6, 8])], dtype=object)

In [90]:
review_sents[1]

[['room',
  'spacious',
  'spacious',
  'bedroom',
  'see',
  'windowwhen',
  'go',
  'caught',
  'typhoon',
  'fortunately',
  'room',
  'occupy',
  'doubledecked',
  'windowsthe',
  'room',
  'modern',
  'technologicaloverall',
  'pretty',
  'good']]

In [80]:
# with open('review_labels.dat','wb') as fp:
#     review_labels.dump(fp)
# with open('review_sents.dat','wb') as fp:
#     review_sents.dump(fp)

In [81]:
# with open('review_labels.lst','rb') as fp:
#     review_labels = np.load(fp)
# with open('review_sents.lst','rb') as fp:
#     review_sents = np.load(fp)

make W

In [72]:
review_cnt = 0
for rev_lst in review.values():
    review_cnt += len(rev_lst)

In [73]:
D = review_cnt # 리뷰수
K = len(aspect_terms) # aspect 수
N = len(voca_freq_dic2) # 단어수
D*K*N

25327762980

In [74]:
# W = np.ndarray((D,K,N),dtype = 'int16') # 메모리에 다 안올라감

In [109]:
# del W

In [110]:
W = {} # k로 나눠 딕셔너리 만들자
for i in range(K):
    W['w_'+str(i)] = np.ndarray((D,N),dtype = 'int16')

In [123]:
K

10

In [ ]:
# for d,rev in enumerate(review_labels):
#     for sent_idx,sent_labels in enumerate(rev):
#         for label in sent_labels:
#             review_sents[d][sent_idx]
#             W[d][label]

In [116]:
len(review_sents)

595387

In [ ]:
# multiprocessing 필요

In [117]:
def kth_count(sent_labels, sents,k,freq_dic_kth):
    for idx,label in enumerate(sent_labels):
        if k in label: # kth aspect이면
            for word in sents[idx]:
                if word not in freq_dic_kth:
                    continue
                freq_dic_kth[word] += 1
        

def make_kth_mat(k):
    global W,D # word freq 담을 전역변수 W. kth aspect 의 freq은 w_k (np.array)에 담는다/ 리뷰개수 D
    for d in tqdm.tqdm(range(D)):# 리뷰개수만큼 돌면서
        freq_dic_kth = dict.fromkeys(voca_freq_dic2.keys(),0) # kth freq dic 초기화
        kth_count(review_labels[d],review_sents[d],k,freq_dic_kth) # freq_dic_kth 을 완성
        words,freqs = zip(*sorted(freq_dic_kth.items(),key = lambda kv:kv[0])) # freq_dic_kth를 사전순으로 정렬해 단어와 freq를 각각 뽑는다
        W['w_'+str(k)][d] = np.array(freqs,dtype='int16')


In [118]:
make_kth_mat(0)

  1%|          | 5505/595387 [00:26<59:27, 165.36it/s]  

KeyboardInterrupt: 